In [1]:
import time
import cv2
import os
import datetime
import csv

import serial
from threading import Thread

In [7]:
# 비디오 들어오나 테스트
cap=cv2.VideoCapture(1)
if not cap.isOpened():
    print("error")
    exit()
print(cap.get(cv2.CAP_PROP_FPS))
print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
while(True):
    ret,cam=cap.read()
    if ret==True:
        cv2.imshow("cam",cam)
        if cv2.waitKey(1) & 0xFF == 27: # esc 키를 누르면 닫음
            cv2.destroyAllWindows()
            cap.release()
            break

30.0
-1.0


# 웹캠으로 프레임 캡처하기

In [6]:
def video():
    init_time = 0
    cap = cv2.VideoCapture(0)
    print(cap.get(cv2.CAP_PROP_FPS))
    print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = 30
    imgs=[]
    msec=[]
    while(cap.isOpened()):
            ret,frame=cap.read()
            
            if ret ==True :
                cv2.imshow('frame', frame)
                if init_time==0:
                    init_time=time.time()
                    init_frame=cap.get(cv2.CAP_PROP_POS_MSEC)
                curr_time = datetime.datetime.now()
                curr_frame=cap.get(cv2.CAP_PROP_POS_MSEC)-init_frame
                print("프레임위치",curr_frame,"현재시간",datetime.datetime.now())# 시작부터 경과시간(msec)
                
                if curr_frame>3000:
                    cap.release() # 멈추기
                    cv2.destroyAllWindows()
                    break
                imgs.append(frame)
                msec.append(curr_frame*0.001)
                
            else :
                cap.release() # 멈추기
                cv2.destroyAllWindows()
                print("error")
                break
            if cv2.waitKey(1) & 0xFF == 27: # esc 키를 누르면 닫음
                cv2.destroyAllWindows()
                cap.release()
    print(len(imgs))
    
    
    
    count=1
    for i in range(len(imgs)):
        cv2.imwrite(images_path+'frame_%d_%s.png'%(count,msec[i]), imgs[i], [cv2.IMWRITE_PNG_COMPRESSION, 0])
        count=count+1

            

In [11]:
path_num = 100
images_path ='D:\\prlab\\ysg\\rppg\\new\\/images/%d/'%path_num 
if not os.path.isdir(images_path):   # 폴더 있는지 확인하고 없으면 생성   
    print("no dir")
    os.mkdir(images_path)
video()

no dir
30.0
-1.0
프레임위치 0.0 현재시간 2021-04-07 14:30:54.663622
프레임위치 33.33329999446869 현재시간 2021-04-07 14:30:54.687474
프레임위치 66.66670000553131 현재시간 2021-04-07 14:30:54.718981
프레임위치 100.0 현재시간 2021-04-07 14:30:54.751828
프레임위치 133.3332999944687 현재시간 2021-04-07 14:30:54.784170
프레임위치 166.6667000055313 현재시간 2021-04-07 14:30:54.817106
프레임위치 200.0 현재시간 2021-04-07 14:30:54.849879
프레임위치 233.3332999944687 현재시간 2021-04-07 14:30:54.882239
프레임위치 266.6667000055313 현재시간 2021-04-07 14:30:54.914319
프레임위치 300.0 현재시간 2021-04-07 14:30:54.944850
프레임위치 333.3332999944687 현재시간 2021-04-07 14:30:54.976784
프레임위치 366.6667000055313 현재시간 2021-04-07 14:30:55.023610
프레임위치 400.0 현재시간 2021-04-07 14:30:55.057890
프레임위치 433.3332999944687 현재시간 2021-04-07 14:30:55.089480
프레임위치 466.6667000055313 현재시간 2021-04-07 14:30:55.120133
프레임위치 500.0 현재시간 2021-04-07 14:30:55.151713
프레임위치 533.3332999944687 현재시간 2021-04-07 14:30:55.184108
프레임위치 566.6667000055313 현재시간 2021-04-07 14:30:55.216221
프레임위치 600.0 현재시간 2021-04-07 14:30:55.247745
프레임위치

# ppg 측정 연동
- pyserial 연결은 관리자모드의 cmd여야 connection가능

In [2]:


class Ubpulse:
    '''
    https://github.com/ubpulse/ubpulse-360-340-320/blob/master/LXD23_ubpulse3xx_CommunicationSpec.pdf
    '''
    def __init__(self, serialPort, serialBaud, dataNumBytes):
        self.isConnected = False
        self.port = serialPort
        self.baud = 115200
        self.dataNumBytes = min(4096, dataNumBytes)
        self.rawData = bytearray(dataNumBytes)
        self.isRun = True
        self.isReceiving = False
        self.thread = None

        self.sync_after = False
        self.packet_tx_idx = 0
        self.data_prev = 0

        self.PUD0 = 0
        self.PCDT_PUD2_CRD = 0
        self.PC = 0  # PacketCount
        self.PUD1 = 0
        self.PCD = 0  # PacketCyclicData

        self.ppg_upper = 0
        self.ppg_lower = 0
        self.pi_upper = 0
        self.pi_lower = 0

        self.ComDeviceID = 0
        self.ppg = []
        self.real_time = []
        self.ppi = 0
        self.bpm = 0
        self.pi = 0

        print('Trying to connect to: ' + str(serialPort) + ' at ' + str(serialBaud) + ' BAUD.')
        try:
            self.serialConnection = serial.Serial(serialPort, serialBaud, timeout=1/256)
            self.isConnected = True
            print('Connected to ' + str(serialPort) + ' at ' + str(serialBaud) + ' BAUD.')
        except:
            print("Failed to connect with " + str(serialPort) + ' at ' + str(serialBaud) + ' BAUD.')
            raise Exception("Connection failed")

    def readSerialStart(self):
        if isinstance(self.thread, type(None)):
            self.thread = Thread(target=self.backgroundThread)
            self.thread.daemon = True
            self.thread.start()
            # Block till we start receiving values
            while not self.isReceiving:
                time.sleep(0.1)

    def backgroundThread(self):  # retrieve data
        time.sleep(1.0)  # give some buffer time for retrieving data
        self.serialConnection.reset_input_buffer()
        while self.isRun:
            self.serialConnection.readinto(self.rawData)
            self.isReceiving = True

            for i in range(len(self.rawData)):
                data_crnt = self.rawData[i]
                if self.data_prev == 255 and data_crnt == 254:
                    self.sync_after = True
                    self.packet_tx_idx = 0
                self.data_prev = data_crnt

                if self.sync_after:
                    self.packet_tx_idx += 1

                    if self.packet_tx_idx > 1:
                        if self.packet_tx_idx == 2:
                            self.PUD0 = data_crnt
                            self.ppi = (self.PUD1 & 0x07) * 256 + self.PUD0
                        elif self.packet_tx_idx == 3:
                            self.PCDT_PUD2_CRD = data_crnt
                        elif self.packet_tx_idx == 4:
                            self.PC = data_crnt
                        elif self.packet_tx_idx == 5:
                            self.PUD1 = data_crnt
                        elif self.packet_tx_idx == 6:
                            self.PCD = data_crnt
                            if self.PC == 2:
                                self.bpm = self.PCD
                            elif self.PC == 3:
                                self.pi_upper = self.PCD
                            elif self.PC == 4:
                                self.pi_lower = self.PCD
                                self.pi = ((self.pi_upper & 0x0F) * 256 + self.pi_lower) * 0.01
                            elif self.PC == 30:
                                self.ComDeviceID = self.PCD
                        elif self.packet_tx_idx == 7:
                            self.ppg_upper = data_crnt
                        elif self.packet_tx_idx == 8:
                            self.ppg_lower = data_crnt
                            self.ppg.append((self.ppg_upper & 0x0F) * 256 + self.ppg_lower)
                            self.real_time.append(str(datetime.datetime.now()))
                        elif self.packet_tx_idx > 8:
                            self.sync_after = False

    def flush(self):
        self.serialConnection.flush()

    def close(self):
        self.isRun = False
        self.serialConnection.close()
        print('Disconnected...')


def video(serial_port,images_path,cppg_path,timeout):
    init_time = 0
    cap = cv2.VideoCapture(1)
    print(cap.get(cv2.CAP_PROP_FPS))
    print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = 30
    imgs=[]
    rppg_msec=[]
    serial_port.readSerialStart() # 시작 시 cppg기록도 시작(background thread시작)
    while(cap.isOpened()):
            ret,frame=cap.read()
            
            if ret ==True :
                cv2.imshow('frame', frame)
                if init_time==0:
                    init_time=time.time()
                    init_frame=cap.get(cv2.CAP_PROP_POS_MSEC)
                    
                    
                curr_time = datetime.datetime.now()
                curr_frame=cap.get(cv2.CAP_PROP_POS_MSEC)-init_frame
                print("프레임위치(sec)",curr_frame*0.001,"현재시간",datetime.datetime.now())# 시작부터 경과시간(msec)
                
                if curr_frame>(timeout*1000):
                    serial_port.close() # cppg기록 stop
                    cap.release() # 멈추기
                    cv2.destroyAllWindows()
                    break
                imgs.append(frame)
                rppg_msec.append(curr_frame)
                
            else :
                serial_port.close()
                cap.release() # 멈추기
                cv2.destroyAllWindows()
                print("error")
                break
            if cv2.waitKey(1) & 0xFF == 27: # esc 키를 누르면 닫음
                serial_port.close()
                cv2.destroyAllWindows()
                cap.release()
    print(len(imgs))
    
    
    # rppg img저장
    count=1
    for i in range(len(imgs)):
        cv2.imwrite(images_path+'frame_%d_%s.png'%(count,rppg_msec[i]), imgs[i], [cv2.IMWRITE_PNG_COMPRESSION, 0])
        print("save",images_path,'frame_%d_%s.png'%(count,rppg_msec[i]))
        count=count+1
    
    # cppg pulse.csv저장
    with open(cppg_path, 'w', newline='') as f:
        wr = csv.writer(f)
        wr.writerow(serial_port.ppg)
        wr.writerow(serial_port.real_time)
        f.close()

def cppg_test(serial_port, cppg_path, timeout):
    init_time = 0
    curr_time = 0
    while True:
        if curr_time == 0:
            init_time = time.time()
            serial_port.readSerialStart()  # starts background thread
            
        curr_time = time.time()
        
        if curr_time - init_time > timeout:
            serial_port.serialConnection.close()
            print(curr_time - init_time)
            break
    print(serial_port.real_time) 
    with open(cppg_path, 'w', newline='') as f:
        wr = csv.writer(f)
        wr.writerow(serial_port.ppg)
        wr.writerow(serial_port.real_time)
        f.close()


if __name__ == '__main__':
    path_num =11 
    cppg_path = "D:\\prlab\\ysg\\rppg\\rppg_HRV\\data\\cppg\\cppg%d.csv"%path_num ##ppg신호 저장할 경로
    images_path ='D:\\prlab\\ysg\\rppg\\rppg_HRV\\images\\%d/'%path_num 
    if not os.path.isdir(images_path):
        print("no dir")# 폴더 있는지 확인하고 없으면 생성                                                   
        os.mkdir(images_path)
    
    portName = 'COM3'
    baudRate = 9600 # 115200
    dataNumBytes = 8  # number of bytes of 1 data point
    sp = Ubpulse(portName, baudRate, dataNumBytes)  # initializes all required variables
    
    
    video(sp,images_path,cppg_path,660) # timeout은 sec
    #cppg_test(sp,cppg_path,timeout=10)
    #cppg_video_test(sp, cppg_path, images_path)

no dir
Trying to connect to: COM3 at 9600 BAUD.
Connected to COM3 at 9600 BAUD.
30.0
-1.0
프레임위치(sec) 0.0 현재시간 2021-06-17 13:24:49.512939
프레임위치(sec) 0.033333300000056625 현재시간 2021-06-17 13:24:49.531345
프레임위치(sec) 0.06666669999994337 현재시간 2021-06-17 13:24:49.563860
프레임위치(sec) 0.1 현재시간 2021-06-17 13:24:49.596879
프레임위치(sec) 0.13333330000005664 현재시간 2021-06-17 13:24:49.629129
프레임위치(sec) 0.16666669999994338 현재시간 2021-06-17 13:24:49.661584
프레임위치(sec) 0.2 현재시간 2021-06-17 13:24:49.692772
프레임위치(sec) 0.23333330000005664 현재시간 2021-06-17 13:24:49.724684
프레임위치(sec) 0.26666669999994336 현재시간 2021-06-17 13:24:49.755913
프레임위치(sec) 0.3 현재시간 2021-06-17 13:24:49.790561
프레임위치(sec) 0.3333333000000566 현재시간 2021-06-17 13:24:49.820194
프레임위치(sec) 0.3666666999999434 현재시간 2021-06-17 13:24:49.868056
프레임위치(sec) 0.4 현재시간 2021-06-17 13:24:49.899345
프레임위치(sec) 0.43333330000005665 현재시간 2021-06-17 13:24:49.931546
프레임위치(sec) 0.46666669999994337 현재시간 2021-06-17 13:24:49.965887
프레임위치(sec) 0.5 현재시간 2021-06-17 13:24:49.996513

프레임위치(sec) 4.733333300000057 현재시간 2021-06-17 13:24:54.220171
프레임위치(sec) 4.766666699999943 현재시간 2021-06-17 13:24:54.269392
프레임위치(sec) 4.8 현재시간 2021-06-17 13:24:54.299166
프레임위치(sec) 4.8333333000000565 현재시간 2021-06-17 13:24:54.331477
프레임위치(sec) 4.866666699999944 현재시간 2021-06-17 13:24:54.363615
프레임위치(sec) 4.9 현재시간 2021-06-17 13:24:54.395915
프레임위치(sec) 4.933333300000057 현재시간 2021-06-17 13:24:54.428249
프레임위치(sec) 4.966666699999943 현재시간 2021-06-17 13:24:54.460110
프레임위치(sec) 5.0 현재시간 2021-06-17 13:24:54.491936
프레임위치(sec) 5.033333300000057 현재시간 2021-06-17 13:24:54.523089
프레임위치(sec) 5.066666699999944 현재시간 2021-06-17 13:24:54.555637
프레임위치(sec) 5.1000000000000005 현재시간 2021-06-17 13:24:54.587854
프레임위치(sec) 5.133333300000057 현재시간 2021-06-17 13:24:54.621313
프레임위치(sec) 5.1666666999999435 현재시간 2021-06-17 13:24:54.667318
프레임위치(sec) 5.2 현재시간 2021-06-17 13:24:54.699658
프레임위치(sec) 5.233333300000057 현재시간 2021-06-17 13:24:54.731750
프레임위치(sec) 5.266666699999943 현재시간 2021-06-17 13:24:54.764795
프레임위치(sec) 5.3 현

프레임위치(sec) 9.633333300000057 현재시간 2021-06-17 13:24:59.131283
프레임위치(sec) 9.666666699999944 현재시간 2021-06-17 13:24:59.163324
프레임위치(sec) 9.700000000000001 현재시간 2021-06-17 13:24:59.195682
프레임위치(sec) 9.733333300000057 현재시간 2021-06-17 13:24:59.227248
프레임위치(sec) 9.766666699999943 현재시간 2021-06-17 13:24:59.259495
프레임위치(sec) 9.8 현재시간 2021-06-17 13:24:59.291335
프레임위치(sec) 9.833333300000056 현재시간 2021-06-17 13:24:59.323515
프레임위치(sec) 9.866666699999943 현재시간 2021-06-17 13:24:59.356701
프레임위치(sec) 9.9 현재시간 2021-06-17 13:24:59.387643
프레임위치(sec) 9.933333300000056 현재시간 2021-06-17 13:24:59.419162
프레임위치(sec) 9.966666699999944 현재시간 2021-06-17 13:24:59.467964
프레임위치(sec) 10.0 현재시간 2021-06-17 13:24:59.499362
프레임위치(sec) 10.033333300000058 현재시간 2021-06-17 13:24:59.534349
프레임위치(sec) 10.066666699999944 현재시간 2021-06-17 13:24:59.563508
프레임위치(sec) 10.1 현재시간 2021-06-17 13:24:59.595756
프레임위치(sec) 10.133333300000057 현재시간 2021-06-17 13:24:59.627733
프레임위치(sec) 10.166666699999944 현재시간 2021-06-17 13:24:59.659770
프레임위치(sec) 10

프레임위치(sec) 14.333333300000056 현재시간 2021-06-17 13:25:03.820139
프레임위치(sec) 14.366666699999943 현재시간 2021-06-17 13:25:03.866941
프레임위치(sec) 14.4 현재시간 2021-06-17 13:25:03.899369
프레임위치(sec) 14.433333300000056 현재시간 2021-06-17 13:25:03.934151
프레임위치(sec) 14.466666699999944 현재시간 2021-06-17 13:25:03.965399
프레임위치(sec) 14.5 현재시간 2021-06-17 13:25:03.995989
프레임위치(sec) 14.533333300000058 현재시간 2021-06-17 13:25:04.027535
프레임위치(sec) 14.566666699999944 현재시간 2021-06-17 13:25:04.059586
프레임위치(sec) 14.6 현재시간 2021-06-17 13:25:04.090880
프레임위치(sec) 14.633333300000057 현재시간 2021-06-17 13:25:04.124137
프레임위치(sec) 14.666666699999944 현재시간 2021-06-17 13:25:04.155841
프레임위치(sec) 14.700000000000001 현재시간 2021-06-17 13:25:04.187451
프레임위치(sec) 14.733333300000057 현재시간 2021-06-17 13:25:04.220050
프레임위치(sec) 14.766666699999943 현재시간 2021-06-17 13:25:04.267610
프레임위치(sec) 14.8 현재시간 2021-06-17 13:25:04.299585
프레임위치(sec) 14.833333300000056 현재시간 2021-06-17 13:25:04.331079
프레임위치(sec) 14.866666699999943 현재시간 2021-06-17 13:25:04.365525
프레

프레임위치(sec) 19.23333330000006 현재시간 2021-06-17 13:25:08.733322
프레임위치(sec) 19.266666699999945 현재시간 2021-06-17 13:25:08.765616
프레임위치(sec) 19.3 현재시간 2021-06-17 13:25:08.795301
프레임위치(sec) 19.333333300000056 현재시간 2021-06-17 13:25:08.827801
프레임위치(sec) 19.366666699999943 현재시간 2021-06-17 13:25:08.858534
프레임위치(sec) 19.400000000000002 현재시간 2021-06-17 13:25:08.890706
프레임위치(sec) 19.433333300000058 현재시간 2021-06-17 13:25:08.924255
프레임위치(sec) 19.466666699999944 현재시간 2021-06-17 13:25:08.955765
프레임위치(sec) 19.5 현재시간 2021-06-17 13:25:08.987848
프레임위치(sec) 19.533333300000056 현재시간 2021-06-17 13:25:09.021515
프레임위치(sec) 19.566666699999942 현재시간 2021-06-17 13:25:09.067193
프레임위치(sec) 19.6 현재시간 2021-06-17 13:25:09.099325
프레임위치(sec) 19.633333300000057 현재시간 2021-06-17 13:25:09.130888
프레임위치(sec) 19.666666699999944 현재시간 2021-06-17 13:25:09.163389
프레임위치(sec) 19.7 현재시간 2021-06-17 13:25:09.195802
프레임위치(sec) 19.73333330000006 현재시간 2021-06-17 13:25:09.227781
프레임위치(sec) 19.766666699999945 현재시간 2021-06-17 13:25:09.259515
프레임위

...

save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19319_643933.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19320_643966.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19321_644000.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19322_644033.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19323_644066.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19324_644100.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19325_644133.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19326_644166.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19327_644200.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19328_644233.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19329_644266.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19330_644300.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19331_644333.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19332_644366.6667.png
save D:\prlab\ysg\

save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19439_647933.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19440_647966.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19441_648000.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19442_648033.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19443_648066.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19444_648100.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19445_648133.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19446_648166.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19447_648200.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19448_648233.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19449_648266.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19450_648300.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19451_648333.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19452_648366.6667.png
save D:\prlab\ysg\

save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19557_651866.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19558_651900.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19559_651933.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19560_651966.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19561_652000.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19562_652033.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19563_652066.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19564_652100.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19565_652133.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19566_652166.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19567_652200.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19568_652233.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19569_652266.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19570_652300.0.png
save D:\prlab\ysg\rpp

save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19676_655833.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19677_655866.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19678_655900.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19679_655933.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19680_655966.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19681_656000.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19682_656033.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19683_656066.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19684_656100.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19685_656133.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19686_656166.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19687_656200.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19688_656233.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19689_656266.6667.png
save D:\prlab\ysg\

save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19796_659833.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19797_659866.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19798_659900.0.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19799_659933.3333.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19800_659966.6667.png
save D:\prlab\ysg\rppg\rppg_HRV\images\11/ frame_19801_660000.0.png


In [6]:
ser=serial.Serial('COM3', 9600, timeout=1/256)
ser.close()